In [56]:
!pip install psycopg2-binary


In [57]:
#import libraries

import pandas as pd
import psycopg2

In [58]:
# DB connection setup using hardcoded credentials (for onboarding only)
conn = psycopg2.connect(
    dbname="neondb",
    user="neondb_owner",
    password="npg_CeS9fJg2azZD",
    host="ep-falling-glitter-a5m0j5gk-pooler.us-east-2.aws.neon.tech",
    port="5432",
    sslmode="require"
)
cur = conn.cursor()

In [59]:
#test query
query = "SELECT  * FROM nyc_schools.high_school_directory ;"
df = pd.read_sql(query, conn)
df.head()

/var/folders/f2/z3f134vj17zg0vm9xq6gmy4h0000gn/T/ipykernel_1682/4212029641.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,dbn,school_name,borough,building_code,phone_number,fax_number,grade_span_min,grade_span_max,expgrade_span_min,expgrade_span_max,...,number_programs,Location 1,Community Board,Council District,Census Tract,Zip Codes,Community Districts,Borough Boundaries,City Council Districts,Police Precincts
0,27Q260,Frederick Douglass Academy VI High School,Queens,Q465,718-471-2154,718-471-2890,9.0,12,NaN,NaN,...,1,"{'latitude': '40.601989336', 'longitude': '-73...",14,31,100802,20529,51,3,47,59
1,21K559,Life Academy High School for Film and Music,Brooklyn,K400,718-333-7750,718-333-7775,9.0,12,NaN,NaN,...,1,"{'latitude': '40.593593811', 'longitude': '-73...",13,47,306,17616,21,2,45,35
2,16K393,Frederick Douglass Academy IV Secondary School,Brooklyn,K026,718-574-2820,718-574-2821,9.0,12,NaN,NaN,...,1,"{'latitude': '40.692133704', 'longitude': '-73...",3,36,291,18181,69,2,49,52
3,08X305,Pablo Neruda Academy,Bronx,X450,718-824-1682,718-824-1663,9.0,12,NaN,NaN,...,1,"{'latitude': '40.822303765', 'longitude': '-73...",9,18,16,11611,58,5,31,26
4,03M485,Fiorello H. LaGuardia High School of Music & A...,Manhattan,M485,212-496-0700,212-724-5748,9.0,12,NaN,NaN,...,6,"{'latitude': '40.773670507', 'longitude': '-73...",7,6,151,12420,20,4,19,12


In [60]:
# query
query = "SELECT * FROM nyc_schools.school_demographics ;"
df = pd.read_sql(query, conn)
df.head()


/var/folders/f2/z3f134vj17zg0vm9xq6gmy4h0000gn/T/ipykernel_1682/4218662837.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,dbn,Name,schoolyear,fl_percent,frl_percent,total_enrollment,prek,k,grade1,grade2,...,black_num,black_per,hispanic_num,hispanic_per,white_num,white_per,male_num,male_per,female_num,female_per
0,01M015,P.S. 015 ROBERTO CLEMENTE,20052006,89.4,NaN,281,15,36,40,33,...,74,26.3,189,67.3,5,1.8,158,56.2,123,43.8
1,01M015,P.S. 015 ROBERTO CLEMENTE,20062007,89.4,NaN,243,15,29,39,38,...,68,28.0,153,63.0,4,1.6,140,57.6,103,42.4
2,01M015,P.S. 015 ROBERTO CLEMENTE,20072008,89.4,NaN,261,18,43,39,36,...,77,29.5,157,60.2,7,2.7,143,54.8,118,45.2
3,01M015,P.S. 015 ROBERTO CLEMENTE,20082009,89.4,NaN,252,17,37,44,32,...,75,29.8,149,59.1,7,2.8,149,59.1,103,40.9
4,01M015,P.S. 015 ROBERTO CLEMENTE,20092010,,96.5,208,16,40,28,32,...,67,32.2,118,56.7,6,2.9,124,59.6,84,40.4


 <h2>🧮 School Distribution</h2>	

How many schools are there in each borough?



In [61]:

query = """
SELECT 
    borough,
    COUNT(*) AS num_schools
FROM nyc_schools.high_school_directory
GROUP BY borough
ORDER BY num_schools DESC;
"""
df = pd.read_sql(query, conn)
df.head()


/var/folders/f2/z3f134vj17zg0vm9xq6gmy4h0000gn/T/ipykernel_1682/4045226098.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,borough,num_schools
0,Brooklyn,121
1,Bronx,118
2,Manhattan,106
3,Queens,80
4,Staten Island,10


 <h2>🎓 Language Learners

</h2>	

What is the average % of English Language Learners (ELL) per borough?



In [64]:

query = """


SELECT 
    h.borough,
    ROUND(AVG(d.ell_percent)::numeric, 2) as avg_ell_percent
FROM nyc_schools.high_school_directory AS h
JOIN nyc_schools.school_demographics AS d   
  ON h.dbn = d.dbn
GROUP BY 
    h.borough
ORDER BY 
    avg_ell_percent DESC;


"""
df = pd.read_sql(query, conn)
df.head()

/var/folders/f2/z3f134vj17zg0vm9xq6gmy4h0000gn/T/ipykernel_1682/1990649943.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,borough,avg_ell_percent
0,Manhattan,7.57


 <h2>🎓 🔗School supporting special needs </h2>	

Using the data from the school demographics and high school directory, write a query to find the top 3 schools in each borough with the highest percentage of special education students (sped_percent)


In [65]:
query = """

WITH ranked AS (
  SELECT
      h.school_name,
      h.borough,
      d.sped_percent,
      RANK() OVER (
          PARTITION BY h.borough
          ORDER BY d.sped_percent DESC
      ) AS rank
  FROM nyc_schools.high_school_directory AS h
  JOIN nyc_schools.school_demographics AS d
    ON h.dbn = d.dbn          
  WHERE d.sped_percent IS NOT NULL
)
SELECT 
    school_name,
    borough,
    sped_percent
FROM ranked
WHERE rank <= 3
ORDER BY borough, rank;
 """
df = pd.read_sql(query, conn)
df.head()

/var/folders/f2/z3f134vj17zg0vm9xq6gmy4h0000gn/T/ipykernel_1682/2282017704.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,school_name,borough,sped_percent
0,East Side Community School,Manhattan,28.8
1,East Side Community School,Manhattan,27.7
2,East Side Community School,Manhattan,26.7
